In [27]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import html
import numpy as np
import re

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly.offline import iplot
from sklearn import preprocessing
import os
from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
import re
import math


# Funciones


In [29]:
def limpiar_resultado(resultado):
    resultado = resultado.replace(' ','')
    resultado = resultado.replace('\r','')
    resultado = resultado.replace('\n','')
    return resultado

In [30]:
Lista_URLs_finales = ['https://www.bdfutbol.com/es/t/t2000-01.html',
'https://www.bdfutbol.com/es/t/t2001-02.html',
'https://www.bdfutbol.com/es/t/t2002-03.html',
'https://www.bdfutbol.com/es/t/t2003-04.html',
'https://www.bdfutbol.com/es/t/t2004-05.html',
'https://www.bdfutbol.com/es/t/t2005-06.html',
'https://www.bdfutbol.com/es/t/t2006-07.html',
'https://www.bdfutbol.com/es/t/t2007-08.html',
'https://www.bdfutbol.com/es/t/t2008-09.html',
'https://www.bdfutbol.com/es/t/t2009-10.html',
'https://www.bdfutbol.com/es/t/t2010-11.html',
'https://www.bdfutbol.com/es/t/t2011-12.html',
'https://www.bdfutbol.com/es/t/t2012-13.html',
'https://www.bdfutbol.com/es/t/t2013-14.html',
'https://www.bdfutbol.com/es/t/t2014-15.html',
'https://www.bdfutbol.com/es/t/t2015-16.html',
'https://www.bdfutbol.com/es/t/t2016-17.html',
'https://www.bdfutbol.com/es/t/t2017-18.html',
'https://www.bdfutbol.com/es/t/t2018-19.html',
'https://www.bdfutbol.com/es/t/t2019-20.html',
'https://www.bdfutbol.com/es/t/t2020-21.html']

In [31]:
Lista_URLs_finales = ['https://www.bdfutbol.com/es/t/t2000-01.html']

# JORNADA

In [32]:
def jornada(soup):
    sopa = soup.find(class_="col-md-12 font-weight-bold text-center text-clar pt-1").text
    sopa_limpia = limpiar_resultado(sopa)
    pattern1 = re.compile(r'Jornada\d\d?')
    match = re.search(pattern1, sopa_limpia)
    jornada = match.group(0)
    return jornada


In [33]:
def fecha(soup):
    sopa = soup.find(class_="col-md-12 font-weight-bold text-center text-clar pt-1").text
    sopa_limpia = limpiar_resultado(sopa)
    pattern1 = re.compile(r'\d\d/\d\d/\d\d\d\d')
    match = re.search(pattern1, sopa_limpia)
    jornada = match.group(0)
    return jornada


# MARCADOR

In [34]:
def resultado(soup):
    marcador = soup.find(class_='row align-self-center text-dark').text
    resultado = limpiar_resultado(marcador)
    return resultado

# Equipo Local y Visitante


In [35]:
def equipo_local(soup):
    equipo =  soup.find(class_='col text-blanc partit-equip').text
    equipo = equipo.replace(' ','')
    equipo = equipo.replace('\r','')
    equipo = equipo.replace('\n','')
    return equipo

In [36]:
def equipo_visitante(soup):
    equipo =  soup.find(class_='col text-blanc partit-equip text-right').text
    equipo = equipo.replace(' ','')
    equipo = equipo.replace('\r','')
    equipo = equipo.replace('\n','')
    return equipo

# Arbitro

In [37]:
def arbitro(soup):
    arbitro =  soup.find(class_="col-sm-4 d-none d-sm-block text-clar").text
    pattern = re.compile(r'\s.*\s?$')
    match = re.search(pattern, arbitro)
    x = match.group(0)
    x = x.replace('\n','')
    x = x[:-1]
    return x

# Amarillas

In [38]:
def buscar_local(soup,clase):
    datos_titulares = soup.find_all('div', class_="col-6 pl-0 pl-md-2 pr-0 p2-md-2")
    x = datos_titulares[0]
    z = x.find_all(class_=clase)
    valores_titulares = len(z)
    datos_supletentes = soup.find_all('div', class_='col-6 pl-0 pl-md-2')
    r = datos_supletentes[0]
    h = r.find_all(class_=clase)
    valores_suplentes = len(h)
    valores_totales = valores_suplentes+valores_titulares
    return valores_totales


In [39]:
def buscar_visitante(soup,clase):
    datos_titulares = soup.find_all('div', class_="col-6 pr-0 pr-md-2 pl-0 pl-md-2")
    x = datos_titulares[0]
    z = x.find_all(class_=clase)
    valores_titulares = len(z)
    datos_supletentes = soup.find_all('div', class_='col-6 pr-0 pr-md-2')
    r = datos_supletentes[0]
    h = r.find_all(class_=clase)
    valores_suplentes = len(h)
    valores_totales = valores_suplentes+valores_titulares
    return valores_totales

> Todo el dato

In [40]:
def datos_goles_locales(soup):
    soup_goles = soup.find_all('div', class_="text-clar text-right")
    x = soup_goles[0]
    goleadores=[]
    minutos=[]
    long = len(x.find_all('div'))
    for i in range(long):
        frase = x.find_all('div')[i].text
        pattern = re.compile(r"\d\d?(\W\d)?'")
        match = re.search(pattern, frase)
        mins = match.group(0)
        minutos.append(mins)
        if '(pp)' in frase: 
            gole = 'Propia Puerta'
            goleadores.append(gole)
        else:
            #pattern1 = re.compile(r'[a-zA-Z]\w*\s?')
            pattern1 = re.compile(r'[a-zA-Z]\w*\s([a-zA-Z]\w*\s)?')
            match = re.search(pattern1, frase)
            gole = match.group(0)
            goleadores.append(gole)
    return minutos, goleadores

In [41]:
def datos_goles_visitantes(soup):
    soup_goles = soup.find_all('div', class_="text-clar text-left")
    x = soup_goles[0]
    goleadores=[]
    minutos=[]
    long = len(x.find_all('div'))
    for i in range(long):
        frase = x.find_all('div')[i].text
        pattern = re.compile(r"\d\d?(\W\d)?'")
        match = re.search(pattern, frase)
        mins = match.group(0)
        minutos.append(mins)
        if '(pp)' in frase: 
            gole = 'Propia Puerta'
            goleadores.append(gole)
        else:
            #pattern1 = re.compile(r'[a-zA-Z]\w*\s?')
            pattern1 = re.compile(r'[a-zA-Z]\w*\s([a-zA-Z]\w*\s)?')
            match = re.search(pattern1, frase)
            gole = match.group(0)
            goleadores.append(gole)
    return minutos, goleadores


# Corregir valores en columnas

In [42]:
def corregir_diccionario(df,columna, dict):
    df[columna] =  df[columna].replace(dict)
    return df 

# SACAR COLUMNAS

# VARIABLE TARGET

In [43]:
def crear_target(df):
    Target = []
    for a in range(len(df)): 
        if df.loc[a,'Goles Local'] > df.loc[a,'Goles Visitante']:
            Target.append('L')
        elif df.loc[a,'Goles Local'] == df.loc[a,'Goles Visitante']:
            Target.append('E')
        else: 
            Target.append('V')
    df.insert(9,'Target',Target,True)
    return df

# DICCIONARIOS y LISTAS

In [44]:
dict_jornada = {'Jornada1': 1, 'Jornada2': 2, 'Jornada3': 3, 'Jornada4': 4, 'Jornada5': 5,
       'Jornada6': 6, 'Jornada7': 7, 'Jornada8': 8, 'Jornada9': 9, 'Jornada10': 10,
       'Jornada11': 11, 'Jornada12': 12, 'Jornada13': 13, 'Jornada14': 14, 'Jornada15': 15,
       'Jornada16': 16, 'Jornada17': 17, 'Jornada18': 18, 'Jornada19': 19, 'Jornada20': 20,
       'Jornada21': 21, 'Jornada22': 22, 'Jornada23': 23, 'Jornada24': 24, 'Jornada25': 25,
       'Jornada26': 26, 'Jornada27': 27, 'Jornada28': 28, 'Jornada29': 29, 'Jornada30': 30,
       'Jornada31': 31, 'Jornada32': 32, 'Jornada33':33, 'Jornada34': 34, 'Jornada35':35,
       'Jornada36': 36, 'Jornada37': 37, 'Jornada38': 38}

In [45]:
lista_jornadas = []
lista_columnas = np.arange(0,38)
for a in range(1,39):
    lista_jornadas.append(a)

dict_columnas = dict(zip(lista_columnas,lista_jornadas))
dict_columnas

{0: 1,
 1: 2,
 2: 3,
 3: 4,
 4: 5,
 5: 6,
 6: 7,
 7: 8,
 8: 9,
 9: 10,
 10: 11,
 11: 12,
 12: 13,
 13: 14,
 14: 15,
 15: 16,
 16: 17,
 17: 18,
 18: 19,
 19: 20,
 20: 21,
 21: 22,
 22: 23,
 23: 24,
 24: 25,
 25: 26,
 26: 27,
 27: 28,
 28: 29,
 29: 30,
 30: 31,
 31: 32,
 32: 33,
 33: 34,
 34: 35,
 35: 36,
 36: 37,
 37: 38}

In [46]:
lista_jornadas_idayvuelta = np.arange(1,20)
lista_columnas_idayvuelta = np.arange(0,19)

dict_columnas_casa_visitante = dict(zip(lista_columnas_idayvuelta,lista_jornadas_idayvuelta))
dict_columnas_casa_visitante

{0: 1,
 1: 2,
 2: 3,
 3: 4,
 4: 5,
 5: 6,
 6: 7,
 7: 8,
 8: 9,
 9: 10,
 10: 11,
 11: 12,
 12: 13,
 13: 14,
 14: 15,
 15: 16,
 16: 17,
 17: 18,
 18: 19}

# FUNCIONES COLUMNAS

In [47]:
def crear_tabla_modelo1(df,nombre,tipo):
    equipo = []
    team = str(nombre)
    if tipo == 'IdayVuelta':
        for a in range(len(df)): 
            if df.loc[a,'Equipo Local'] == team and df.loc[a, 'Goles Local']>df.loc[a,'Goles Visitante'] or df.loc[a,'Equipo Visitante'] == team and df.loc[a,'Goles Visitante']>df.loc[a,'Goles Local']:
                equipo.append(3)
            elif df.loc[a,'Equipo Local'] == team and df.loc[a, 'Goles Local']==df.loc[a,'Goles Visitante'] or df.loc[a,'Equipo Visitante'] == team and df.loc[a,'Goles Visitante']==df.loc[a,'Goles Local']:
                equipo.append(1)
            elif df.loc[a,'Equipo Local'] == team and df.loc[a, 'Goles Local']<df.loc[a,'Goles Visitante'] or df.loc[a,'Equipo Visitante'] == team and df.loc[a,'Goles Visitante']<df.loc[a,'Goles Local']:
                equipo.append(0)

    elif tipo == 'Local':
        for a in range(len(df)): 
            if df.loc[a,'Equipo Local'] == team and df.loc[a, 'Goles Local']>df.loc[a,'Goles Visitante']:
                equipo.append(3)
            elif df.loc[a,'Equipo Local'] == team and df.loc[a, 'Goles Local']==df.loc[a,'Goles Visitante']:
                equipo.append(1)
            elif df.loc[a,'Equipo Local'] == team and df.loc[a, 'Goles Local']<df.loc[a,'Goles Visitante']:
                equipo.append(0)     
                
    elif tipo == 'Visitante':
        for a in range(len(df)): 
            if df.loc[a,'Equipo Visitante'] == team and df.loc[a, 'Goles Local']>df.loc[a,'Goles Visitante']:
                equipo.append(0)
            elif df.loc[a,'Equipo Visitante'] == team and df.loc[a, 'Goles Local']==df.loc[a,'Goles Visitante']:
                equipo.append(1)
            elif df.loc[a,'Equipo Visitante'] == team and df.loc[a, 'Goles Local']<df.loc[a,'Goles Visitante']:
                equipo.append(3)

    elif tipo == 'Goles Local a favor':
        for a in range(len(df)): 
            if df.loc[a,'Equipo Local'] != team:
                continue
            goles = df.loc[a,'Goles Local']
            equipo.append(goles)

    elif tipo == 'Goles Visitante a favor':
        for a in range(len(df)): 
            if df.loc[a,'Equipo Visitante'] != team:
                continue
            goles = df.loc[a,'Goles Visitante']
            equipo.append(goles)

    elif tipo == 'Goles Local en contra':
        for a in range(len(df)): 
            if df.loc[a,'Equipo Local'] != team:
                continue
            goles = df.loc[a,'Goles Visitante']
            equipo.append(goles)


    elif tipo == 'Goles Visitante en contra':
        for a in range(len(df)): 
            if df.loc[a,'Equipo Visitante'] != team:
                continue
            goles = df.loc[a,'Goles Local']
            equipo.append(goles)

    elif tipo == 'Tarjetas Amarillas':
        for a in range(len(df)): 
            if df.loc[a,'Equipo Local'] == team:
                n_tarjetas = df.loc[a, 'Amarillas Local']
                equipo.append(n_tarjetas)
            elif df.loc[a,'Equipo Visitante'] == team:
                n_tarjetas = df.loc[a, 'Amarillas Visitante']
                equipo.append(n_tarjetas)
    
    elif tipo == 'Tarjetas Rojas':
        for a in range(len(df)): 
            if df.loc[a,'Equipo Local'] == team:
                n_tarjetas = df.loc[a, 'Rojas Local']
                equipo.append(n_tarjetas)
            elif df.loc[a,'Equipo Visitante'] == team:
                n_tarjetas = df.loc[a, 'Rojas Visitante']
                equipo.append(n_tarjetas)
    
    
    return equipo


In [48]:
def puntuacion_equipo_modelo2(df,tipom1, tipom2):
    lista_equipos = []
    lista_puntos = []
    for a in df['Equipo Local'].unique():
        lista_equipos.append(a)
        puntos = crear_tabla_modelo1(df,a,tipom1)
        j = 0
        equipo = []
        if tipom2 != 'Clasificacion':
            lista_puntos.append(puntos)
        elif tipom2 == 'Clasificacion':
            for i in range(0,len(puntos)):
                j+=puntos[i]
                equipo.append(j)
            lista_puntos.append(equipo)

    dict_puntos = dict(zip(lista_equipos, lista_puntos))

    if tipom1 == 'IdayVuelta' or tipom1 == 'Tarjetas Amarillas' or tipom1 =='Tarjetas Rojas':
        tabla = pd.DataFrame(dict_puntos, index=dict_columnas.values())
        tabla = tabla.transpose()
    elif tipom1 != 'IdayVuelta':
        tabla = pd.DataFrame(dict_puntos, index=dict_columnas_casa_visitante.values())
        tabla = tabla.transpose()
    return tabla

In [49]:
def crear_tabla_ranking(tabla_clasificacion):
    orden_n = []
    ranking = np.arange(1,21)
    for column in tabla_clasificacion: 
        tabla_clasificacion.sort_values(by=column,ascending=False,inplace=True)
        orden = list(tabla_clasificacion.index)
        dict_clasificacion=dict(zip(orden,ranking))
        orden_n.append(dict_clasificacion)

    diccionario_final = dict(zip(lista_jornadas, orden_n))
    tabla_ranking = pd.DataFrame(diccionario_final)
    return tabla_ranking

In [50]:
def crear_tabla_racha(tabla):
    puntuaciones_jornada_tran = tabla.transpose()
    racha_5 = pd.DataFrame(puntuaciones_jornada_tran.rolling(5).sum())
    tabla_racha = racha_5.transpose()
    return tabla_racha

In [51]:
def crear_listas_dataframe(dataframe, campo, nombre_tabla):
    lista_tabla = []
    for a in range(len(dataframe)):
        jornada = dataframe.loc[a,'Jornada']
        jornada = jornada-1
        if jornada == 0:
                puntuacion_L = 0
                lista_tabla.append(puntuacion_L)
        elif jornada != 0:
                team = dataframe.loc[a,campo]
                puntuacion_L = nombre_tabla.loc[team,jornada]
                lista_tabla.append(puntuacion_L)

    return lista_tabla

In [52]:
def promedio_lista(dataframe,tipo1,tipo2, columna_df,tabla):
    promedio_lista = []
    if tipo2 == '19':
        for a in range(len(dataframe)):
            jor_nmb = dataframe.loc[a,'Jornada']
            jornada = jor_nmb%2
            if jor_nmb ==1 or jor_nmb ==2:
                promedio = 0
                promedio_lista.append(promedio)

            elif jornada == 1 and jor_nmb != 1:
                numero = math.floor(jor_nmb/2)
                puntos_posibles = numero*3
                team = dataframe.loc[a,columna_df]
                puntos_L = tabla.loc[team,numero]
                if tipo1 =='Promedio Clasificacion':
                    promedio = puntos_L/puntos_posibles
                    promedio_lista.append(round(promedio,3))
                elif tipo1 =='Promedio Goles':
                    promedio = puntos_L/numero
                    promedio_lista.append(round(promedio,3))

            else: 
                numero = jor_nmb/2 -1
                puntos_posibles = numero*3
                team = dataframe.loc[a,columna_df]
                puntos_L = tabla.loc[team,numero]
                if tipo1 =='Promedio Clasificacion':
                    promedio = puntos_L/puntos_posibles
                    promedio_lista.append(round(promedio,3))
                elif tipo1 =='Promedio Goles':
                    promedio = puntos_L/numero
                    promedio_lista.append(round(promedio,3))
    if tipo2 == '38':
        for a in range(len(dataframe)):
            jornada = dataframe.loc[a,'Jornada']-1
            if jornada == 0:
                    promedio_lista.append(0)
            elif jornada != 0:
                    team = dataframe.loc[a,columna_df]
                    amarillas_L = tabla.loc[team,jornada]
                    promedio_amarillas = round(amarillas_L/jornada,2)
                    promedio_lista.append(promedio_amarillas)

    return promedio_lista

# FUNCIONES ARBITROS

In [53]:
def arbitros(df, nombre, tipo1, tipo2,columna):
    equipo = []
    Fechas = []
    arbitro = str(nombre)
    for a in range(len(df)): 
        if df.loc[a,columna] == arbitro:
            if columna == 'Arbitro':
                n_tarjetas = df.loc[a, tipo1] + df.loc[a,tipo2]
            else:
                n_tarjetas = df.loc[a, tipo1]
            fecha_partido = df.loc[a, 'Fecha']
            Fechas.append(fecha_partido)
            equipo.append(n_tarjetas)
    return equipo, Fechas

In [54]:
def crear_tabla_tarjetas(tarjetas,fechas):

    arbitro_df = pd.DataFrame(list(zip(fechas,tarjetas,np.arange(1,len(tarjetas)+1))), columns=['Fechas','Tarjetas','Nº Partidos'])
    arbitro_df.sort_values(by='Fechas',inplace=True)
    arbitro_df['Tarjetas'] = arbitro_df['Tarjetas'].cumsum()
    arbitro_df['Promedio']=arbitro_df['Tarjetas']/arbitro_df['Nº Partidos']
    arbitro_df['Posibles puntos'] = arbitro_df['Nº Partidos']*3

    return arbitro_df


In [55]:
def crear_lista_arbitros_suma(df,tipo1, tipo2,columna): 
    lista_promedio = []
    for a in range(len(df)):
        arbitro_partido = df.loc[a,columna]
        fecha_partido = df.loc[a,'Fecha']
        tarjetas, fechas = arbitros(df,arbitro_partido,tipo1,tipo2,columna)
        x = crear_tabla_tarjetas(tarjetas, fechas)
        indice = x.index[x['Fechas']==fecha_partido][0]-1
        if indice == -1:
            promedio = 0
            lista_promedio.append(round(promedio,3))
        elif indice != -1:
            promedio = x.iloc[indice,3]
            lista_promedio.append(round(promedio,3))


    return lista_promedio

In [56]:
def crear_lista_portres(df,tipo1,columna): 
    lista_promedio = []
    for a in range(len(df)):
        arbitro_partido = df.loc[a,columna]
        fecha_partido = df.loc[a,'Fecha']
        tarjetas, fechas = crear_tabla_modeloprom(df,arbitro_partido,tipo1)
        x = crear_tabla_tarjetas(tarjetas, fechas)
        indice = x.index[x['Fechas']==fecha_partido][0]-1
        if indice == -1:
            promedio = 0
            lista_promedio.append(round(promedio,3))
        elif indice != -1:
            promedio = x.iloc[indice,1]/x.iloc[indice,4]
            lista_promedio.append(round(promedio,3))


    return lista_promedio

# Lanzar Código

In [58]:
lista_partidos = []
n= 0

for i in Lista_URLs_finales:
    r = requests.get(i)
    content = r.content
    soup = bs(content, 'lxml')
    n+=1
    lista_temporada = []


    futbol_grid = soup.find_all(class_='partiteq d-flex align-items-center pl-2 pr-2')

    lista_URLS = []
    for futbol in futbol_grid:
        try: 
            X = futbol.find('a')['href']
            lista_URLS.append(X)
        except: 
            print('No hay link')

    part1_URL = 'https://www.bdfutbol.com/es'
    URLs_definitivas_2021 = []

    for i in lista_URLS:
        URL_final = part1_URL + i[2:]
        URLs_definitivas_2021.append(URL_final)

    URLs_final = []
    for i in URLs_definitivas_2021:
        if i in URLs_final:
            continue
        URLs_final.append(i)

    for page in range(len(URLs_final)):
        
        r = requests.get(URLs_final[page])
        soup_partido = bs(r.text, 'lxml')
        try:
            dia = fecha(soup_partido)
        except:
            dia = None
        try: 
            J = jornada(soup_partido)
        except:
            J = None
        try:
            TeamLocal = equipo_local(soup_partido)
        except:
            TeamLocal = None
        try:
            TeamVisit = equipo_visitante(soup_partido)
        except:
            TeamVisit = None
        try: 
            Resultado = resultado(soup_partido)
        except: 
            Resultado = None
        try:
            GolesLocal = buscar_local(soup_partido, 'G') + buscar_local(soup_partido,'GP') +  buscar_visitante(soup_partido,'GPP')
        except:
            GolesLocal = None
        try:
            GolesVisitante = buscar_visitante(soup_partido, 'G') + buscar_visitante(soup_partido,'GP') + buscar_local(soup_partido,'GPP')
        except:
            GolesVisitante = None
        try:
            PenaltisLocal = buscar_local(soup_partido, 'GP')
        except:
            PenaltisLocal = None
        try: 
            PenaltisVisitante = buscar_visitante(soup_partido, 'GP')
        except:
            PenaltisVisitante = None
        try:
            Referee = arbitro(soup_partido)
        except:
            Referee = None
        try:
            Amarillas_Local = buscar_local(soup_partido, 'TG') + buscar_local(soup_partido, 'TG2')
        except:
            Amarillas_Local = None
        try:
            Amarillas_Visitante = buscar_visitante(soup_partido, 'TG') + buscar_visitante(soup_partido, 'TG2')
        except:
            Amarillas_Visitante=None
        try:
            RojasLocal = buscar_local(soup_partido, 'TV')
        except:
            RojasLocal = None
        try:
            RojasVisitante = buscar_visitante(soup_partido, 'TV')
        except:
            RojasVisitante=None
        try:
            TarjetasTotales = Amarillas_Local + Amarillas_Visitante + RojasLocal + RojasVisitante
        except: 
            TarjetasTotales = None
        try:
            minutos_local, goleadores_local = datos_goles_locales(soup_partido)
        except:
            minutos_local=None
        try:
            minutos_visitante, goleadores_visitante = datos_goles_visitantes(soup_partido)
        except:
            minutos_visitante=None
            
        data = {'Temporada': n,
                'Fecha': dia,
                'Jornada': J,
                'Equipo Local': TeamLocal,
                'Equipo Visitante': TeamVisit,
                'Resultado': Resultado,
                'Goles Local': GolesLocal,
                'Goles Visitante': GolesVisitante,
                'Penaltis Local': PenaltisLocal,
                'Penaltis Visitante': PenaltisVisitante,
                'Arbitro': Referee,
                'Amarillas Local': Amarillas_Local,
                'Amarillas Visitante': Amarillas_Visitante,
                'Rojas Local': RojasLocal,
                'Rojas Visitante': RojasVisitante,
                'Minutos Goles L': minutos_local,
                'Goleadores L': goleadores_local,
                'Minutos Goles V': minutos_visitante,
                'Goleadores V': goleadores_visitante}

        print(dia,J,TeamLocal,TeamVisit)
        lista_partidos.append(data)
        lista_temporada.append(data)

    dataframe = pd.DataFrame(lista_temporada)
    dataframe = crear_target(dataframe)
    dataframe = corregir_diccionario(dataframe,columna = 'Jornada', dict = dict_jornada)
    dataframe.sort_values(by='Jornada', inplace=True)
    dataframe.reset_index(inplace=True, drop=True)
  
    
    # Sacamos tablas
    puntuaciones_jornada =puntuacion_equipo_modelo2(df = dataframe, tipom1='IdayVuelta', tipom2='diferente')
    clasificacion =puntuacion_equipo_modelo2(df = dataframe, tipom1='IdayVuelta', tipom2='Clasificacion')
    tabla_ranking = crear_tabla_ranking(clasificacion)
    Racha_forma = crear_tabla_racha(puntuaciones_jornada)
    clasificacion_Local =puntuacion_equipo_modelo2(df = dataframe, tipom1='Local', tipom2='Clasificacion')
    clasificacion_Visitante =puntuacion_equipo_modelo2(df = dataframe, tipom1='Visitante', tipom2='Clasificacion')
    Goles_local =puntuacion_equipo_modelo2(df = dataframe, tipom1='Goles Local a favor', tipom2='Clasificacion')
    Goles_visitante =puntuacion_equipo_modelo2(df = dataframe, tipom1='Goles Visitante a favor', tipom2='Clasificacion')
    Goles_local_encontra =puntuacion_equipo_modelo2(df = dataframe, tipom1='Goles Local en contra', tipom2='Clasificacion')
    Goles_visitante_encontra =puntuacion_equipo_modelo2(df = dataframe, tipom1='Goles Visitante en contra', tipom2='Clasificacion')
    Amarillas =puntuacion_equipo_modelo2(df = dataframe, tipom1='Tarjetas Amarillas', tipom2='Clasificacion')
    Rojas =puntuacion_equipo_modelo2(df = dataframe, tipom1='Tarjetas Rojas', tipom2='Clasificacion')

    # Creamos Columnas 
    dataframe['PTOS L']= crear_listas_dataframe(dataframe, campo ='Equipo Local', nombre_tabla=clasificacion)
    dataframe['PTOS V'] = crear_listas_dataframe(dataframe, campo ='Equipo Visitante', nombre_tabla=clasificacion)
    dataframe['POSC L'] = crear_listas_dataframe(dataframe, campo ='Equipo Local', nombre_tabla=tabla_ranking)
    dataframe['POSC V'] = crear_listas_dataframe(dataframe, campo ='Equipo Visitante', nombre_tabla=tabla_ranking)
    dataframe['FORMA L'] = crear_listas_dataframe(dataframe, campo ='Equipo Local', nombre_tabla=Racha_forma)
    dataframe['FORMA V'] = crear_listas_dataframe(dataframe, campo ='Equipo Visitante', nombre_tabla=Racha_forma)
    dataframe['Pr PTOS L']  = promedio_lista(dataframe,tipo1='Promedio Clasificacion',tipo2='19', columna_df ='Equipo Local',tabla=clasificacion_Local)
    dataframe['Pr PTOS V']  = promedio_lista(dataframe,tipo1='Promedio Clasificacion',tipo2='19', columna_df ='Equipo Visitante',tabla=clasificacion_Visitante)
    dataframe['Pr G AF L'] =  promedio_lista(dataframe,tipo1='Promedio Goles',tipo2='19', columna_df ='Equipo Local',tabla=Goles_local)
    dataframe['Pr G AF V'] =  promedio_lista(dataframe,tipo1='Promedio Goles',tipo2='19', columna_df ='Equipo Visitante',tabla=Goles_visitante)
    dataframe['Pr G EC L'] = promedio_lista(dataframe,tipo1='Promedio Goles',tipo2='19', columna_df ='Equipo Local',tabla=Goles_local_encontra)
    dataframe['Pr G EC V'] = promedio_lista(dataframe,tipo1='Promedio Goles',tipo2='19', columna_df ='Equipo Visitante',tabla=Goles_visitante_encontra)
    dataframe['Pr TA L'] = promedio_lista(dataframe, tipo1='Promedio goles', tipo2='38', columna_df ='Equipo Local', tabla=Amarillas)
    dataframe['Pr TA V'] = promedio_lista(dataframe, tipo1='Promedio goles', tipo2='38', columna_df ='Equipo Visitante', tabla=Amarillas)
    dataframe['Pr TR L'] = promedio_lista(dataframe, tipo1='Promedio goles', tipo2='38', columna_df ='Equipo Local', tabla=Rojas)
    dataframe['Pr TR V'] = promedio_lista(dataframe, tipo1='Promedio goles', tipo2='38', columna_df ='Equipo Visitante', tabla=Rojas)
    dataframe['Pr TA ARB'] = crear_lista_arbitros_suma(dataframe,'Amarillas Local','Amarillas Visitante','Arbitro')
    dataframe['Pr TR ARB'] = crear_lista_arbitros_suma(dataframe,'Rojas Local','Rojas Visitante','Arbitro')

    dataframe.to_csv(f"../Raw/Temporadas200{n}.csv", index = False)
    

09/09/2000 Jornada1 Barcelona Málaga
09/09/2000 Jornada1 DeportivodeLaCoruña AthleticClub
09/09/2000 Jornada1 RealSociedad RacingdeSantander
09/09/2000 Jornada1 RealMadrid Valencia
09/09/2000 Jornada1 Zaragoza Espanyol
10/09/2000 Jornada1 LasPalmas Alavés
10/09/2000 Jornada1 Mallorca Valladolid
10/09/2000 Jornada1 Numancia Oviedo
10/09/2000 Jornada1 Osasuna CeltadeVigo
10/09/2000 Jornada1 Villarreal RayoVallecano
16/09/2000 Jornada2 AthleticClub Barcelona
16/09/2000 Jornada2 Málaga RealMadrid
16/09/2000 Jornada2 RacingdeSantander DeportivodeLaCoruña
16/09/2000 Jornada2 Valencia Mallorca
16/09/2000 Jornada2 Valladolid Numancia
17/09/2000 Jornada2 CeltadeVigo RealSociedad
17/09/2000 Jornada2 Alavés Villarreal
17/09/2000 Jornada2 Espanyol Osasuna
17/09/2000 Jornada2 RayoVallecano Zaragoza
17/09/2000 Jornada2 Oviedo LasPalmas
23/09/2000 Jornada3 Barcelona RacingdeSantander
23/09/2000 Jornada3 LasPalmas Valladolid
23/09/2000 Jornada3 RealSociedad DeportivodeLaCoruña
23/09/2000 Jornada3 Real